In [ ]:
!pip install tensorflow-gpu

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
#####################################################################
########################## DATA PROCESSING ##########################
#####################################################################
labels = ['fruit_healthy','fruit_brownspot','fruit_woodiness']
dataset = pd.read_csv('/content/gdrive/My Drive/Tuks/COS711/Assignment3/Train.csv')
dataset['class'] = dataset['class'].astype(int)
dataset['Image_ID'] = dataset['Image_ID'].apply(lambda x: "{}{}".format(x, '.jpg'))
target = dataset.pop('class')
#target = target.truncate(before=0, after=999) #crop for parameter optimisation
identifiers = dataset.pop('Image_ID')
#identifiers = identifiers.truncate(before=0, after=999) #crop for parameter optimisation
image_size = 512
image_set = []
for img in identifiers:
  input_arr = cv2.imread('/content/gdrive/My Drive/Tuks/COS711/Assignment3/Train_Images/'+img)
  image_set.append(input_arr)
image_set = np.array(image_set).reshape(-1, image_size, image_size, 3)
X_train, X_test, y_train, y_test = train_test_split(image_set, target, test_size=0.1)
#####################################################################
############################ MODEL SETUP ############################
#####################################################################
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3))
#####################################################################
########################## MODEL CREATION ###########################
#####################################################################
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#####################################################################
########################## MODEL FITTING ############################
#####################################################################
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))